In [2]:
import torch # Delo z matriki z CPU/GPU podporo
from torchvision import datasets, transforms # Predprocesiranje slik
from torch.utils.data import DataLoader # Obravnava podatkov po manjših sklopih

In [4]:
data_dir = 'Podatki_split'

In [5]:
# Definicija transofrmacije za konsistentnost podatkov (CNN želi 224x224, normalizacija)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [6]:
# Naloži slike iz map
batch_size = 32

train_dataset = datasets.ImageFolder(root=f'{data_dir}/timeofday/train', transform=transform)
val_dataset   = datasets.ImageFolder(root=f'{data_dir}/timeofday/val', transform=transform)
test_dataset  = datasets.ImageFolder(root=f'{data_dir}/timeofday/test', transform=transform)

In [7]:
# Razdeli slike po sklopih, boljša učinkovitost
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [9]:
print("Oznake/Labels:", train_dataset.classes)
print("Število slik v učnem sklopu:", len(train_dataset))

Oznake/Labels: ['day', 'night']
Število slik v učnem sklopu: 6430


#### Training a Classifier: https://docs.pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
##### Docs: https://www.geeksforgeeks.org/introduction-convolution-neural-network/

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
class TimeOfDayCNN(nn.Module):
    def __init__(self):
        super(TimeOfDayCNN, self).__init__()

        # Dve konvolucijske plasti
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)

        # Popolnoma povezana/Fully connected plast
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x): # Pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### Testiranje obnašanja podatkov v modelu

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TimeOfDayCNN().to(device)

dataiter = iter(train_loader)
images, labels = next(dataiter)

print("Oblika vhodnih slik:", images.shape)
print("Oznake:", labels)

images = images.to(device)
outputs = model(images)

print("Oblika izhoda modela:", outputs.shape)  # želim [32, 2] za 32 slik in 2 različne oznake

Oblika vhodnih slik: torch.Size([32, 3, 224, 224])
Oznake: tensor([0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 0])
Oblika izhoda modela: torch.Size([32, 2])


In [18]:
# Loss funkcija in optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Treniranje mreže

In [ ]:
num_epochs = 5 # Ni isto kot epizoda!, epoch - SL, episode - RL

for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()                  # Tukaj se izvede backpropagation
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Povprečna izguba: {avg_loss:.4f}")